# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [ ]:
# Data Analytics and Visualization Bootcamp 
# Student Name: Thet Win
# Module 6 - Python APIs (Vacation API - Hotel locations)
# Modified Date: June 9, 2024
# 
# This program searches for nearby hotels in each city within a dataset using APIs from geoapify.com.
# It returns a log of hotel list along with city names and the country name.
# The program then produces a visual of the locations of the hotels on a map. 

In [5]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import os
from pathlib import Path

# Import API key
from api_keys import geoapify_key

# Set up relative Path using Current Working Directory
cwd=Path.cwd().parent

# File to output
outputfile = os.path.join(cwd,'output_data','cities.csv')
output_data_fldr = os.path.join(cwd,'output_data')
starter_code_fldr = os.path.join(cwd,'starter_code')

In [6]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv(outputfile, index_col="City_ID")

# Display sample data
city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
City_ID,,,,,,,,,
0,queenstown,-31.8976,26.8753,10.48,52,0,3.35,ZA,1717954899
1,port-aux-francais,-49.3500,70.2167,6.77,99,100,16.92,TF,1717954899
2,yellowknife,62.4560,-114.3525,12.29,73,75,5.14,CA,1717954899
3,pasuquin,18.3328,120.6177,28.00,78,100,1.94,PH,1717954900
4,ust-nera,64.5667,143.2000,5.74,48,80,1.93,RU,1717954900


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [57]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,  
    size = "Humidity",
    alpha = 0.5,
    color = "City"  
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [38]:
# Narrow down cities that fit criteria and drop any results with null values

# A max temperature lower than 27 degrees but higher than 21
# Wind speed less than 4.5 m/s
# Zero cloudiness

city_data_filtered_df = city_data_df.loc[(city_data_df['Max Temp'] < 27) & (city_data_df['Max Temp'] > 21) & (city_data_df['Wind Speed'] < 4.5)]

# Drop any rows with null values
city_data_filtered_df = city_data_filtered_df.dropna()

# Display sample data
city_data_filtered_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
City_ID,,,,,,,,,
6,uige,-7.6087,15.0613,22.68,51,61,1.02,AO,1717954900
14,kalkaman,51.9525,76.0211,21.65,76,63,4.29,KZ,1717954900
19,purwodadi,-7.0868,110.9158,24.97,90,90,1.34,ID,1717954901
20,bondo,3.8146,23.6866,23.01,85,94,2.34,CD,1717954901
33,weno,7.4515,151.8468,26.97,100,100,0.00,FM,1717954902


### Step 3: Create a new DataFrame called `hotel_df`.

In [46]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_filtered_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
City_ID,,,,,,
6,uige,AO,-7.6087,15.0613,51,
14,kalkaman,KZ,51.9525,76.0211,76,
19,purwodadi,ID,-7.0868,110.9158,90,
20,bondo,CD,3.8146,23.6866,85,
33,weno,FM,7.4515,151.8468,100,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [50]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey":geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
uige - nearest hotel: No hotel found
kalkaman - nearest hotel: No hotel found
purwodadi - nearest hotel: Hotel Kencana
bondo - nearest hotel: No hotel found
weno - nearest hotel: High Tide Hotel
badger - nearest hotel: Clarion Hotel & Suites
argo - nearest hotel: Apollon Hotel
avarua - nearest hotel: Paradise Inn
newman - nearest hotel: No hotel found
benoy-yurt - nearest hotel: No hotel found
sa'dah - nearest hotel: الشارقة مول للمواد الغذائية
luganville - nearest hotel: Beachfront Resort
tazacorte - nearest hotel: App Leyma
sinabang - nearest hotel: No hotel found
san antonio de pale - nearest hotel: No hotel found
hirara - nearest hotel: 古謝そば屋
georgetown - nearest hotel: Page 63 hostel
batouri - nearest hotel: No hotel found
arteche - nearest hotel: No hotel found
lompoc - nearest hotel: Red Roof Inn Lompoc
hawaiian paradise park - nearest hotel: No hotel found
banjar - nearest hotel: Villa Banjar Beach
mistelbach - nearest hotel: Weißes Rössel
kabalo - nearest

,City,Country,Lat,Lng,Humidity,Hotel Name
City_ID,,,,,,
6,uige,AO,-7.6087,15.0613,51,No hotel found
14,kalkaman,KZ,51.9525,76.0211,76,No hotel found
19,purwodadi,ID,-7.0868,110.9158,90,Hotel Kencana
20,bondo,CD,3.8146,23.6866,85,No hotel found
33,weno,FM,7.4515,151.8468,100,High Tide Hotel
...,...,...,...,...,...,...
558,inirida,CO,3.8653,-67.9239,98,Residencias El Turista del Guainia
575,pocatello,US,42.8713,-112.4455,44,Black Swan Inn
578,vanimo,PG,-2.6741,141.3028,92,CBC Guesthouse


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [56]:
%%capture --no-display

# Configure the map plot
map_plot_hotels = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    alpha = 0.5,
    color = "City" ,
    hover_cols = ['Hotel Name', 'Country']    
)

# Display the map
map_plot_hotels

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)